<a href="https://colab.research.google.com/github/andygma567/LLM-experiments/blob/main/4_bit_Falcon_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!nvidia-smi

Sun Jul 16 12:20:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%bash
pip install -U bitsandbytes
pip install -U git+https://github.com/huggingface/transformers.git
pip install -U git+https://github.com/huggingface/accelerate.git
# required for Falcon
pip install -U einops xformers scipy
# an endpoint for serving
pip install gradio

Process is terminated.


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import transformers
import torch
import warnings
# warnings.filterwarnings("ignore")

from transformers import BitsAndBytesConfig

# Load the model

I think cpu offloading with device map is only available for 8bit models and I think in general it's expected that one tries to put everything on gpu

ref:
- https://colab.research.google.com/drive/1VoYNfYDKcKRQRor98Zbf2-9VQTtGJ24k?usp=sharing
- https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu


 This is an error message that I previously had when using device_map="auto"

 Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit
                        the quantized model. If you want to dispatch the model on the CPU or the disk while keeping
                        these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom
                        `device_map` to `from_pretrained`. Check
                        https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                        for more details.

I think the fix is to use a custom device map that puts everything on gpu: see this example maybe https://github.com/huggingface/accelerate/issues/362#issuecomment-1125328460 or passing in `device_map={'': 0}`


In [4]:
HF_ckpt = "tiiuae/falcon-40b-instruct"
drive_ckpt = "tiiuae/falcon-40b-instruct"

In [5]:
%%time
# Test using 4bit configs

tokenizer = AutoTokenizer.from_pretrained(HF_ckpt)

# Use a bnb config here:
# note that using a bnb config negates any bnb args passed in
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# load the model
model = AutoModelForCausalLM.from_pretrained(
    HF_ckpt,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    quantization_config=quantization_config,
    # device_map="auto", # I wonder if using a quantization_config requires me to put everything on gpu
)
# I still need to use a high ram instance because I often don't have enough RAM to offload shards

# When I try to use a V100 then I get this warning message
# I have a warning msg:
# UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
# for the v100 gpu

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-40b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


ImportError: ignored

In [6]:
# check how much memory is used up
!nvidia-smi

Sun Jul 16 12:19:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load a pipeline

In [7]:
%%time
pipeline = transformers.pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
)
generation_config = GenerationConfig(
    max_new_tokens=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

NameError: ignored

In [8]:
%%time
sequences = pipeline(
   "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:",
   generation_config=generation_config
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

NameError: ignored

# Set up a gradio demo

In [ ]:
# import gradio as gr

# # We instantiate the Textbox class
# input_textbox = gr.Textbox(placeholder="Please enter an instruction", lines=2)

# demo = gr.Interface(
#     title=drive_checkpoint, # the path where the model is
#     fn=use_OpenAlpaca,
#     inputs=input_textbox,
#     outputs="text",
#     examples=[["What is an alpaca? How is it different from a llama?"],["How does a car work?"]],
#     cache_examples=True,
#     )

In [ ]:
# demo.launch(share=True) #, debug=True)